In [22]:
import meshio as mio
import numpy as np
from scipy.io import savemat
import meshplot as mp
import torch
import numpy as np
import matplotlib.pyplot as plt
import igl
# from pydec import simplicial_complex, d, delta, whitney_innerproduct

torch.set_printoptions(precision=16)

In [23]:
def vec_mass(rho, V, T):
    Vs = V[T, :]
    vol = torch.linalg.det(Vs[:, 1:4, :] - Vs[:, 0:3, :])
    tmp = rho*vol
    
    mass = torch.sum(tmp)
    
    return mass

# def vec_mass(rho, vol):
#     tmp = rho*vol
#     mass = torch.sum(tmp)
    
#     return mass

In [24]:
def vec_cm(rho, V, T):
    Vs = V[T, :]
    vol = torch.linalg.det(Vs[:, 1:4, :] - Vs[:, 0:3, :])
    tmp = rho*vol
    
    cm = torch.sum(tmp*torch.mean(Vs, axis=1).T, axis=1)
    mass = torch.sum(tmp)
    
    return cm / mass

# def vec_cm(rho, V, T, vol):
#     Vs = V[T, :]
#     vol = torch.linalg.det(Vs[:, 1:4, :] - Vs[:, 0:3, :])
#     tmp = rho*torch.tensor(vol)
    
#     cm = torch.sum(tmp*torch.mean(Vs, axis=1).T, axis=1)
#     mass = torch.sum(tmp)
    
#     return cm / mass

# def vec_cm(rho, V, T, vol):
#     Vs = V[T, :]
#     tmp = torch.tensor(vol)*rho
#     com = torch.zeros(3)
#     for i, t in enumerate(T):
#         com += tmp[i] * torch.mean(V[t], axis=0)

#     com = com/torch.sum(tmp)
    
#     return com

In [25]:
def vec_energy(r, newcm, V, T):
    return torch.sum((newcm - vec_cm(r,V,T))**2);

# def vec_energy(r, newcm, V, T, vol):
#     return torch.sum((newcm - vec_cm(r,V,T, vol))**2);

In [27]:
m = mio.read("../meshes/armadillo-standing/armadillo-hollow-coarse.msh")
# m = mio.read("gargoyle.msh")
V = m.points
T = m.cells[0].data
input_rho = 1000


V = torch.tensor(V, dtype = torch.float64)
T = torch.tensor(T, dtype = torch.int64)

print(V[T,:].shape)

vol = igl.volume(V.numpy(), T.numpy())
# vol = 0.5 *igl.doublearea(V.numpy(), T.numpy())

# err = torch.tensor(vol) - torch.linalg.det(V[T,:][:, 1:4, :] - V[T,:][:, 0:3, :])
# print(np.linalg.norm(err, np.inf))

rt = torch.ones(T.shape[0], dtype = torch.float64)*input_rho
target = vec_cm(rt, V, T).detach()
target_mass = vec_mass(rt, V, T).detach()

print(target.numpy(), target_mass.numpy())

####################################

m = mio.read("../meshes/armadillo-standing/armadillo-hollow-coarser.msh")
# m = mio.read("gargoyle-coarse.msh")
V = m.points
T = m.cells[0].data

V = torch.tensor(V, dtype = torch.float64)
T = torch.tensor(T, dtype = torch.int64)

vol = igl.volume(V.numpy(), T.numpy())
# vol = 0.5 *igl.doublearea(V.numpy(), T.numpy())

torch.Size([7248, 4, 3])
[0.40570231 0.55690153 0.53282308] 160.65093968983737


In [28]:
# L-BFGS
r0 = np.random.rand(T.shape[0])+1
r0 = torch.tensor(r0, requires_grad=True, dtype = torch.float64)
# r0 = torch.rand(T.shape[0], requires_grad=True, dtype = torch.float64)
# target = torch.tensor([0.4,0,0])
print(vec_cm(r0, V, T).detach().numpy())

optimizer = torch.optim.Adam([r0])#,
#                               lr=0.1,
#                               history_size=20,
#                               max_iter=10,
#                               line_search_fn="strong_wolfe")

f = lambda r: vec_energy(r, target, V, T)
vv = []
for i in range(1000):
    if i % 50 == 0:
        print(i)
    optimizer.zero_grad()
    objective = f(r0)
    objective.backward()
    optimizer.step(lambda: f(r0))
    vv.append(objective.item())
    
    if objective.item() < 1e-14:
        break
    
print(vv[-1])
# plt.semilogy(vv)

opt = r0.detach()
opt_mass = vec_mass(opt, V, T).detach()
opt *=  target_mass/opt_mass
opt_mass = vec_mass(opt, V, T).detach()
opt = opt.numpy()

print("opt:", vec_cm(r0, V, T).detach().numpy(), "vs target:", target.numpy())
print("opt:", opt_mass.numpy(), "vs target:", target_mass.numpy())

print("start:", np.sqrt(vv[0]),"opt:",np.sqrt(vv[-1]))
print("min rho:", np.min(opt), "max rho:",np.max(opt))

print(np.linalg.norm(vec_cm(r0, V, T).detach().numpy()-target.numpy()))

[0.40706509 0.5606683  0.52471773]
0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
9.171163040186808e-14
opt: [0.40570234 0.55690162 0.5328228 ] vs target: [0.40570231 0.55690153 0.53282308]
opt: 160.6509396898374 vs target: 160.65093968983737
start: 0.009041153434989844 opt: 3.028392814709942e-07
min rho: 419.308655821762 max rho: 1349.7702746239522
2.976348522023781e-07


In [29]:
np.savetxt("rhos.txt", opt)
!head rhos.txt

1.236734521486551330e+03
6.566906757850043732e+02
1.040158487494471501e+03
8.666240324183106623e+02
7.258217247189123782e+02
9.127904544716410555e+02
6.842616719538505095e+02
8.171577584912782868e+02
8.959478350328574834e+02
1.309390548947835669e+03
